In [87]:
import numpy as np
import pandas as pd
import pydicom as dicom
import cv2
import os

In [84]:
# path to covid-19 dataset from https://github.com/ieee8023/covid-chestxray-dataset
imgpath = 'data/cohen/images'
csvpath = 'data/cohen/metadata.csv'

# path to RSNA data
rsna_img = "data/rsna/images"
# get normal cases
rsna_csv = "data/rsna/stage_2_detailed_class_info.csv"
# to get pneumonia data
rsna_csv2 = "data/rsna/stage_2_train_labels.csv"

# categorize the disiases
mapping = dict()
mapping['COVID-19'] = 'COVID-19'
mapping['SARS'] = 'pneumonia'
mapping['MERS'] = 'pneumonia'
mapping['Streptococcus'] = 'pneumonia'
mapping['Klebsiella'] = 'pneumonia'
mapping['Chlamydophila'] = 'pneumonia'
mapping['Legionella'] = 'pneumonia'
mapping['Normal'] = 'normal'
mapping['Lung Opacity'] = 'pneumonia'
mapping['1'] = 'pneumonia'

In [57]:
metadata = pd.read_csv(csvpath)
views = ["PA", "AP", "AP Supine", "AP semi erect", "AP erect"]
metadata = metadata[metadata.view.isin(views)]

In [70]:
patients1 = {'normal': [], 'pneumonia': [], 'COVID-19': []}
count = {'normal': 0, 'pneumonia': 0, 'COVID-19': 0}
for index, row in metadata.iterrows():
    f = row['finding'].split(',')[0] # take the first finding, for the case of COVID-19, ARDS
    if f in mapping: # 
        count[mapping[f]] += 1
        #entry = [str(row['patientid']), row['filename'], mapping[f], row['view']]
        patients1[mapping[f]].append(row['filename'])

print('Data distribution from covid-chestxray-dataset:')
print(count)

Data distribution from covid-chestxray-dataset:
{'normal': 0, 'pneumonia': 33, 'COVID-19': 218}


In [69]:
csv_normal = pd.read_csv(rsna_csv, nrows=None)
csv_pneu = pd.read_csv(rsna_csv2, nrows=None)
patients2 = {'normal': [], 'pneumonia': []}


for index, row in csv_normal.iterrows():
    if row['class'] == 'Normal':
        patients2['normal'].append(row['patientId'])
        
for index, row in csv_pneu.iterrows():
    if int(row['Target']) == 1:
        patients2['pneumonia'].append(row['patientId'])

In [106]:
try:
    os.mkdir('dataset')
    path_ds = os.path.abspath('dataset')
    os.mkdir(path_ds + '/COVID-19')
    os.mkdir(path_ds + '/normal')
    os.mkdir(path_ds + '/pneumonia')
    print("Directory " , dirName ,  " Created ") 
except FileExistsError:
    print("Directory " , dirName ,  " already exists")

Directory  dataset  Created 


In [101]:
os.path.abspath('dataset/')

'/home/calvus/Projects/Diploma/COVID-19_X-RAY_Detection/dataset'